In [15]:

# This is a file for testing performance of masque
# Run this file on three different machines
# change the variables: role, ip_dict
# Assign each machine a role 
#   and fill in ip address in ip_dict



# You can run these linux commands for setting up the environment
'''
cd ~
sudo git clone https://github.com/dx2102/masque-linux
cd masque-linux
sudo chmod a+rwx -R .

code --install-extension ms-python.python
code --install-extension ms-toolsai.jupyter
code --install-extension github.copilot

sudo apt update
sudo apt install -y python3-pip libssl-dev
pip install aioquic ipykernel
'''



'\ncd ~\nsudo git clone https://github.com/dx2102/masque-linux\ncd masque-linux\nsudo chmod a+rwx -R .\n\ncode --install-extension ms-python.python\ncode --install-extension ms-toolsai.jupyter\ncode --install-extension github.copilot\n\nsudo apt update\nsudo apt install -y python3-pip libssl-dev\npip install aioquic ipykernel\n'

In [16]:
# Imports, Utilities
import time
import os
import socket
import aioquic

def detect_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.connect(("google.com", 80))
    ip = s.getsockname()[0]
    s.close()
    return ip


In [17]:
local_ip = detect_ip()
print("IP of this machine: ", detect_ip())

IP of this machine:  128.110.217.143


In [18]:
# configs:

role = 'client'
assert role in ['client', 'proxy', 'echo']

ip_dict = {
    'client': '',
    'proxy': '',
    'echo': ''
}
assert ip_dict[role] == local_ip

AssertionError: 